#### This report reads the All Incoming Students (Deposited only) and OPS Photo_Data files, Admitted Grad students from Reports and compares GTID in both files to provide photo submission status. IDs are also compared in DB to update photo status, as some IDs with no photos in OPS may have photos in TS.

In [9]:
import pandas as pd
import datetime as dt
from datetime import datetime
currentDateTime = datetime.now().strftime("%m-%d-%Y %H%M%S %p")
#currentDateTime = datetime.now().strftime("%m-%d-%Y")
import warnings
warnings.simplefilter("ignore")

#### Read the FASET student file, OPS file and Reports file and create three data frames-dfu- undergrad, dfp-photo, dfg-grad students file

In [10]:
#Read the FASET student file and create data frames 
dfu = pd.read_csv('C:/Users/FASET.csv',usecols=[6,7,8])
dfp = pd.read_csv ('C:/Users/Photo_Data.csv',usecols=[0,2])
dfg = pd.read_excel('C:/Users/admitted by term.xlsx',usecols=[0,1,2,10],engine="openpyxl")


### Run the following if you want to know details about any dataframes before data cleansing

In [11]:
dfu.shape
#dfg.shape

(360, 3)

In [13]:
#dfu.head()

In [13]:
#dfu.groupby(['Student Type']).count()

In [14]:
dfg.groupby(['Campus']).count()

,ID,Last Name,First Name
Campus,,,
1,122,122,122
3,2,2,2
A,231,231,231
EM,67,67,67
GC,18,18,18
J,53,53,53
O,5299,5299,5299


### Prep OPS data, Undergrad SLATE data and Grad Reports data

In [15]:
#process the OPS photo data
# removes leading 0s from the identifier fieled
dfp['identifier'] = dfp['identifier'].str.lstrip('0')

#renames the identifier field name to GTID
dfp=dfp.rename(columns = {'identifier':'GTID'})

#drop rows with null values for GTID
dfp = dfp[dfp['GTID'].notna()]

# find the size of ops file after data formatting
dfp.shape 

########################################################################################
#Process Undergrad admissions data ##

#filter for only Transfer and First-Year students
dfu = dfu.apply(lambda row: row[dfu['Student Type'].isin(['First-year','Transfer'])])

#if the data type is not same the merge will not work
# convert int64 to object type data
dfu['GTID'] = dfu['GTID'].apply(lambda x: str(x))
dfu['GTID'].astype(int).astype(str)

#########################################################################################
#process Grad admissions data ##

#prep the grad admitted data first
#filtering index for Campuses not in 'A' and 'EM'
#indexes = dfg[ (dfg['Campus'] != 'A') & (dfg['Campus'] != 'EM') ].index
indexes = dfg[ (dfg['Campus'] != 'A') ].index
#droping mutiple rows based on column value
dfg.drop(indexes,inplace=True)

#renames the identifier field name to GTID
dfg=dfg.rename(columns = {'ID':'GTID'})

#if the data type is not same the merge will not work
#dfg['GTID'] = dfg['GTID'].apply(lambda x: str(x))
#dfg['GTID'].astype(int).astype(str)

In [71]:
#dfu.groupby(['Student Type']).count()

### Merge Undergrad, Grad and OPS data. Combine the two data frames for grad and undergrad data

In [7]:
################################################################################
#Merge Undergrad and OPS data
# using merge function by setting how='left' merge two csv files on GTID
#Join OPS file dfp with Undergrad student file dfu
dfu = pd.merge(dfu, dfp, 
                   on='GTID', 
                   how='left')

#replace NaN with Not SUbmitted
dfu['Photo Status'] = dfu['Photo Status'].fillna('Not Submitted')

#select the GTID and Photo Status column only
dfu = dfu.iloc[0:,[0,3]]

#show the Photo Status distribution--
#dfu.groupby(['Photo Status']).count()

###################################################################################
#Merge Grad data and OPS data
# using merge function by setting how='left' merge two csv files on GTID
#Join OPS file dfp with Grad student file dfu
dfg = pd.merge(dfg, dfp, 
                   on='GTID', 
                   how='left')

#replace NaN with Not SUbmitted
dfg['Photo Status'] = dfg['Photo Status'].fillna('Not Submitted')

#select the GTID and Photo Status column only
dfg = dfg.iloc[0:,[0,4]]

#show the Photo Status distribution--
#dfg.groupby(['Photo Status']).count()

dfu['Class'] = 'Undergrad'
dfg['Class'] = 'Grad'

### Combine the two final dataframes to one
frames = [dfu,dfg]
dfc=pd.concat(frames)

#show the Photo Status distribution by Class--
dfc.groupby(['Photo Status','Class']).count()

GTID
Photo Status  Class          
Approved      Grad         92
              Undergrad   105
Denied        Grad          7
              Undergrad     4
Not Submitted Grad        131
              Undergrad   248
Pending       Grad          1
              Undergrad     3

In [31]:
dfc.shape

(595, 3)

### Grab the "Not Submitted"  IDs from the above report (Grad and Undergrad) and compare in DB, if they have photos submitted

In [8]:
# Grab the "Not Submitted" GTIDs from the above report and compare in DB, if they have photos submitted
dfcns=dfc[dfc["Photo Status"]=='Not Submitted']

## Export the file to a folder
dfcns.to_csv(f"C:/Users/Grad_Undergrad_Admitted_OPS_Status-Not_Submiitted-{currentDateTime}.csv", index = False)

### Get the GTIDs from this file and compare in DB to see how many have photos in DB. These are IDs with no photos in OPS with photos in DB

In [9]:
dfcts= pd.read_csv('C:/Users/TS_Approved_Photos.csv')

## update the OPS status by updating the approved status of these IDs in the combined data frame dfc

dfcts['GTID'] = dfcts['GTID'].apply(lambda x: str(x))
dfcts['GTID'].astype(int).astype(str)

dfc = dfc.set_index('GTID')
dfcts = dfcts.set_index('GTID')
dfc.update(dfcts)
dfc.reset_index(inplace=True)

In [10]:
dfc.groupby(['Photo Status','Class']).count()

GTID
Photo Status  Class          
Approved      Grad        146
              Undergrad   113
Denied        Grad          7
              Undergrad     4
Not Submitted Grad         77
              Undergrad   240
Pending       Grad          1
              Undergrad     3

In [11]:
# Grab the "Approved" GTIDs from the above report and compare in DB, if they have photos submitted
dfca=dfc[dfc["Photo Status"]=='Approved']

##create output file - approved photos
dfca.to_csv(f"C:/Users/Grad_Undergrad_Admitted_OPS_Status-Approved-{currentDateTime}.csv", index = False)

## create output- OPS status, update student card production file with the data below
dfc.to_csv(f"C:/Users/Grad_Undergrad_Admitted_OPS_Status-{currentDateTime}.csv", index = False)